In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install

In [11]:
from datasets import load_metric, load_dataset
import torch
from transformers import (
    TrOCRProcessor,
    AutoFeatureExtractor,
    AutoTokenizer,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    HfArgumentParser,
)
import wandb
from dataclasses import dataclass, field
from typing import Optional
import os
from torch.utils.data import Dataset
import pandas as pd

In [ ]:
class OCRDataset(Dataset):
    def __init__(self, df, processor, transforms=lambda x:x, max_target_length=128):
        self.df = df
        self.processor = processor
        self.transforms = transforms
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        #file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = self.df['image'][idx].convert("RGB")
        image = self.transforms(image)
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
encoder = "facebook/deit-base-distilled-patch16-384"
decoder = "asafaya/bert-base-arabic"
model_name = "microsoft/trocr-small-handwritten"

In [ ]:
training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size= 8, #train_args.per_device_train_batch_size,
        per_device_eval_batch_size= 8, #train_args.per_device_eval_batch_size,
        fp16=True,
        adam_beta1=0.9,
        adam_beta2=0.98,
        adam_epsilon=1e-08,
        num_train_epochs=10, #train_args.num_train_epochs,
        weight_decay=0.005,
        learning_rate=5e-5, #train_args.learning_rate,
        seed=42,
        report_to="wandb",
        load_best_model_at_end=True,
        metric_for_best_model="cer",
        do_train=True,
        do_eval=True,
        do_predict=True,
        output_dir = "./"#train_args.output_dir,
    )

In [ ]:
dataset = load_dataset(
        "gagan3012/OnlineKhatt"
)

In [ ]:
if model_name is None:
    tokenizer = AutoTokenizer.from_pretrained(decoder)
    feature_extractor = AutoFeatureExtractor.from_pretrained(encoder)
    processor = TrOCRProcessor(feature_extractor, tokenizer)
    model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder)

else:
    processor = TrOCRProcessor.from_pretrained(model_name)
    model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
        encoder, decoder)

    #fn_kwargs = dict(
    #    processor = processor,
    #)
    #df = dataset.map(preprocess,fn_kwargs=fn_kwargs,remove_columns=["id"])

df_train = pd.DataFrame(dataset['train'])
df_eval = pd.DataFrame(dataset['dev'])
df_pred = pd.DataFrame(dataset['test'])

df_train = df_train.sample(frac=1)
df_eval = df_eval.sample(frac=1)
df_pred = df_pred.sample(frac=1)

transformer = lambda x: x

train_dataset = OCRDataset(df=df_train,
                           processor=processor,
                           max_target_length=128,
                           transforms=transformer)

eval_dataset = OCRDataset(df=df_eval,
                          processor=processor,
                          max_target_length=128,
                          transforms=transformer)

predict_dataset = OCRDataset(df=df_pred,
                             processor=processor,
                             max_target_length=128,
                             transforms=transformer)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")
print(f"Predict dataset size: {len(predict_dataset)}")

model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
!pip install jiwer

In [ ]:
cer_metric = load_metric("cer")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=processor.feature_extractor,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator,
)

In [ ]:
print("Training model")
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
print("Evaluating model")
metrics = trainer.evaluate(metric_key_prefix="eval")
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print("Predicting")
predict_results = trainer.predict(
    predict_dataset,
    metric_key_prefix="predict",
)
metrics = predict_results.metrics
max_predict_samples = (
    data_args.max_predict_samples
    if data_args.max_predict_samples is not None
    else len(predict_dataset)
)
metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

trainer.log_metrics("predict", metrics)
trainer.save_metrics("predict", metrics)

if trainer.is_world_process_zero():
    if training_args.predict_with_generate:
        predictions = processor.batch_decode(
            predict_results.predictions,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )
        predictions = [pred.strip() for pred in predictions]
        output_prediction_file = os.path.join(
            model_args.save_dir, "generated_predictions.txt"
        )
        print(predictions)
        with open(output_prediction_file, "w") as writer:
            writer.write("\n".join(predictions))